# Flower Identifier

In [7]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tqdm as notebook_tqdm

In [8]:
dataset, info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)

In [9]:
train_dataset, test_dataset, val_dataset = dataset['train'], dataset['test'], dataset['validation']

In [10]:
IMG_SIZE = 224

def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize the image to 224x224
    image = tf.cast(image, tf.float32) / 255.0  # Normalize the pixel values to [0, 1]
    return image, label

In [11]:
train_dataset = train_dataset.map(preprocess_image).batch(32)
test_dataset = test_dataset.map(preprocess_image).batch(32)
val_dataset = val_dataset.map(preprocess_image).batch(32)

In [12]:
# Load model
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [13]:
# Freeze the base model
base_model.trainable = False

# Build the final model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(102, activation='softmax')
])

In [14]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=val_dataset
)

Epoch 1/10


2025-03-14 19:49:14.474136: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:387] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608


32/32 ━━━━━━━━━━━━━━━━━━━━ 32s 881ms/step - accuracy: 0.0426 - loss: 4.8337 - val_accuracy: 0.3402 - val_loss: 3.2067
Epoch 2/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 28s 895ms/step - accuracy: 0.6207 - loss: 2.3575 - val_accuracy: 0.6196 - val_loss: 2.0980
Epoch 3/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 30s 947ms/step - accuracy: 0.8646 - loss: 1.2455 - val_accuracy: 0.7029 - val_loss: 1.6118
Epoch 4/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 855ms/step - accuracy: 0.9490 - loss: 0.7616 - val_accuracy: 0.7353 - val_loss: 1.3609
Epoch 5/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 850ms/step - accuracy: 0.9699 - loss: 0.5089 - val_accuracy: 0.7598 - val_loss: 1.2151
Epoch 6/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 851ms/step - accuracy: 0.9888 - loss: 0.3624 - val_accuracy: 0.7686 - val_loss: 1.1207
Epoch 7/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 850ms/step - accuracy: 0.9972 - loss: 0.2703 - val_accuracy: 0.7755 - val_loss: 1.0548
Epoch 8/10
32/32 ━━━━━━━━━━━━━━━━━━━━ 27s 856ms/step - accuracy: 0.9997 - loss: 0.2092 - val_accuracy: 0.789